# Modin一行代码让pandas加速
pandas库是python中最有名的数据分析库，因为dataframe这种易用强大的数据类型，pandas成为数据科学必备套件。pandas可以和很多包联合使用，比如与机器学习sklearn、统计分析statsmodels、可视化searborn&matplotlib等等。下图是近几年python包的使用量趋势，pandas一骑绝尘

![](img/一骑绝尘.png)

但pandas有一个弱点：不适用于大规模数据。

pandas默认只调用电脑单个的CPU进行数据读取和运算，但是当前的电脑大多是4核，甚至8核，而如果常规使用pandas意味着我们没有充分使用电脑的性能。

Modin是为加速pandas而开发的包，可以自动按照电脑CPU数调整运行的核数。

### 安装

In [ ]:
!pip3 install ray
!pip3 install dask
!pip3 install modin

### modin使用
modin的用法与pandas相差无几，只是在导入的时候略有不同。

```python
import modin.pandas as pd
```

现在这个pd就拥有与pandas一样的功能


### 读取速度
说了这么多了，我们先看看pandas导入test.csv文件(107M)的速度。

In [4]:
import time
import pandas as pd


start = time.time()
df = pd.read_csv('data/test.csv')
end = time.time()
print(end-start)

1.611081838607788


pandas导入107M的test.csv耗时1.78s

In [5]:
import time
import modin.pandas as pd


start = time.time()
#test.csv 107M
df = pd.read_csv('data/test.csv')
end = time.time()
print(end-start)

0.7474761009216309


modin只用了0.75s，加快了2.1倍。



### 运算速度
pd.concat是用来连接多个dataframe的操作函数，当我们的df很大时，pd.concat就会变慢。我们在这里实验一下pandas和modin各自的速度

In [39]:
import time
import pandas as pd

df = pd.read_csv('data/test.csv')
start = time.time()
newdf = pd.concat([df for _ in range(100)])
end = time.time()
print(end-start)

10.625022888183594


In [6]:
import time
import modin.pandas as pd

df = pd.read_csv('data/test.csv')
start = time.time()
newdf = pd.concat([df for _ in range(100)])
end = time.time()
print(end-start)

0.7488729953765869


同样的pd.concat操作，modin比pandas快了14倍。我又做了几个对比，制作成表格

|操作|pandas时间|modin时间|提速倍数|
|---|---|---|---|
|pd.read_csv('train.csv')#195M的train.csv|2.58|1.02|2.53|
|pd.concat(\[df for _ in range(100)\])|29.81|1.33|22|
|\[df.describe() for i in range(100)\]|30.8|11.4|2.7|
|\[df.count() for i in range(100)\]|64.16|11.02|5.8|


### 实战技巧
modin目前仍然正在发展中，并不是所有的pandas的函数都能加速。如果加速遇到报错，说明这个操作modin不支持。

默认modin会调用电脑全部的cpu，如果不想全部调用，可以使用ray来限制cpu使用数。


In [4]:
import ray
import warnings
warnings.filterwarnings('ignore')
ray.init(num_cpus=4, ignore_reinit_error=True)
import modin.pandas as pd

2020-01-13 23:25:25,093	ERROR worker.py:1379 -- Calling ray.init() again after it has already been called.


如果待操作的数据文件远大于电脑内存RAM，可以设置

In [7]:
import os
os.environ["MODIN_OUT_OF_CORE"]='true'
import modin.pandas as pd